## 5/1/19

### *m4.xlarge* instance
The m4.xlarge instances have 4 vCPUs, 16GiB memory, and 32 GiB of EBS storage. The following information contain the architecture of the instance, the number of vCPUs, threads per vCPU/core, the processor, and the cache sizes:<br>
`Architecture:          x86_64
CPU(s):                4
On-line CPU(s) list:   0-3
Thread(s) per core:    2
Core(s) per socket:    2
Socket(s):             1
Model name:            Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              46080K`

Need to run `export PYSPARK_PYTHON=python3.5` (for stand-alone instance) or `export PYSPARK_PYTHON=python3.4` (for cluster) in commandline to use python3.4 as default python for Spark Cluster. Spark cluster already has `numpy` and `scipy`.

### Testing sequential code on stand alone *m4.xlarge* instance using Python 3.5
12min and 54seconds to run sequential pipeline with *Genome-Pos-3T3-Unsorted_100_seqs.txt* and *Genome-Pos-3T3-Bot10_100_seqs.txt* input files. Both of these input file were constructed with 75 perfect matching reads and 25 that required ED calculation. About 12 of the ED sequencing reads come in the first 50 lines of each input file to split them up evenly.

Output matches the output I got on my own machine.

### Testing spark code on stand alone *m4.xlarge* instance using Python 3.5 and Spark local mode

I have written the Spark application in the file `spark_implementation.py`.

To run the `spark_implementation.py` code on the input files *Genome-Pos-3T3-Unsorted_100_seqs.txt* and *Genome-Pos-3T3-Bot10_100_seqs.txt* using 2, 3 or 4 cores on a single *m4.xlarge* instance, I modified my `spark_implementation.py` code by including a `[c]` (where $c=2,3,4$) in the code at `SparkConf().setMaster('local[c]')`. 

I used the following command to run my tests:<br>
`spark-submit spark_implementation_local.py -u Genome-Pos-3T3-Unsorted_100_seqs.txt -s Genome-Pos-3T3-Bot10_100_seqs.txt -g Brie_CRISPR_library_with_control_guides.csv -o spark_run_1_core`.

The table below lists the number of cores used in local mode, the elapsed time, and the speed-up when I use 2, 3 or 4 cores.

|Number of cores| Elapsed time |Speed-up|
|---------------|--------------|--------|
|1 |  685 sec | -|
|2 |  352 sec |1.94x|
|3 |  sec ||
|4 |  sec ||

### Why does local mode with 3 or 4 threads specified only give me two tasks?

### Testing spark code on Spark Cluster of m4.xlarge instances

To tune the number of tasks/partitions the input files should be broken up into, I changed the Spark application to be able to take in a user-defined variable for the number of partitions the input files should be split into through command-line; the flag `-n` allows the user to define the number of partitions. This application is saved as `spark_implementation_distributed.py`. Within this application/code, I changed the number of partitions for Spark by adding an argument in `sc.textFile("Genome-Pos-3T3-Unsorted_100_seqs.txt", n)` where `n` is the number of partitions specified by the user through the `-n` flag.

The table below shows my efforts to tune different parameters to improve the speed-up when using up to 8 nodes. The speed-up is compared to the performance of 1 core on 1 node of a m4.xlarge instances. 

I ran the following command:<br>
`spark-submit --num-executors 2 --executor-cores 2 spark_implementation_distributed.py -u Genome-Pos-3T3-Unsorted_100_seqs.txt -s Genome-Pos-3T3-Bot10_100_seqs.txt -g Brie_CRISPR_library_with_control_guides.csv -o spark_run_4_core -n 4`


|Number of worker nodes| Number of cores|Number of tasks| Elapsed time |Speed-up|
|---------------|--------------|--------------|--------|--------------|
|2 | 2 | 4 |  439 sec | 1.56x|
|2 | 2 | 32 |  249 sec | 2.75x|
|2 | 2 | 50 |  243 sec | 2.82x|
|4 | 2 | 32 |   144 sec | 4.76x|
|4 | 2 | 50 |   148sec | 4.63x|

